In [1]:
from Brands import Sonites
from Analyzer import Analyzer
from Solver import Solver
import numpy as np

son = Sonites()
an = Analyzer()
solver = Solver()



d:\Uni\Anno III\UCSD\112\Markstrat\Code\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


./Exports/TeamExport_A46051_Alpha_M_Period 3.xlsx loaded
No weights provided
before
./Exports/TeamExport_A46051_Alpha_M_Period 3.xlsx loaded
after
./Exports/TeamExport_A46051_Alpha_M_Period 3.xlsx loaded
Attributes file:./Attributes/attributes_3.json


In [2]:
df_seg_sem_fc = an.forecast_df(dataframe=son.df_segments_semantic, steps=2)
target_sem = df_seg_sem_fc[(df_seg_sem_fc["Segment"]=="Explorers")&(df_seg_sem_fc["Period"]==5)][['# Features', 'Design Index', 'Battery Life', 'Display Size', 'Proc. Power', 'Price']].values
target_sem = target_sem.flatten()
target_sem

array([4.489999999999991, 2.7399999999999896, 5.42999999999998,
       5.489999999999952, 5.909999999999947, 4.39999999999999],
      dtype=object)

In [3]:
df_seg_mds_fc = an.forecast_df(dataframe=son.df_segments_mds, steps=2)
target_mds = df_seg_mds_fc[(df_seg_mds_fc["Segment"]=="Explorers")&(df_seg_mds_fc["Period"]==5)][['Economy', 'Performance', 'Convenience']].values
target_mds = target_mds.flatten()
target_mds

array([-2.9, 12.199999999999974, -3.8200000000000056], dtype=object)

In [4]:
an.compute_distance_centroids(son.move_semantic.values.flatten(), target_sem, weighted="default")

({'observation': {'centroid': 1.7488281790958737}},
 {'observation': {'centroid': array([0.053990610328636515, 0.06201550387596491, 0.08431703204047547,
          0.20394736842104227, 0.29039301310042503, 0.06581740976645653],
         dtype=object)}},
 {'observation': {'centroid': 0.6159677419354692}},
 {'observation': {'centroid': 0.8727911117316123}},
 {'observation': {'centroid': array([0.22999999999999154, 0.15999999999998948, 0.5000000000000195,
          0.9299999999999526, 1.3299999999999468, 0.31000000000001027],
         dtype=object)}},
 {'observation': {'centroid': 0.07176085783118921}})

In [5]:
an.compute_distance_centroids(son.move_mds.values.flatten(), target_mds, weighted="eq", max_distance_1D=40)

({'observation': {'centroid': 8.55619074121186}},
 {'observation': {'centroid': array([-0.385593220338983, 2.177083333333327, -0.020512820512819052],
         dtype=object)}},
 {'observation': {'centroid': 3.4199999999999893}},
 {'observation': {'centroid': 0.8765020243080888}},
 {'observation': {'centroid': array([1.8199999999999998, 8.359999999999975, 0.0799999999999943],
         dtype=object)}},
 {'observation': {'centroid': 0.5903257641605082}})

In [6]:
son.move_features

,# Features,Design Index,Battery Life,Display Size,Proc. Power,Rec. retail price,Base Cost
MARKET : Sonites,,,,,,,
MOVE,13,6,82,31,83,350,158


In [7]:
from Utils import combined_error_minimum_distance

_ = combined_error_minimum_distance(features=[13, 6, 82, 31, 83, 350], ideal_semantic=target_sem, ideal_mds=target_mds, semantic_weights=son.rel_importance_features,
                                mds_weights=[1/3, 1/3, 1/3], error_weights=[1,1], model=solver)


In [8]:
solver.find_optimum_constrained(ideal_semantic=target_sem.flatten(), ideal_mds=target_mds.flatten(), semantic_weights=son.rel_importance_features)

[I 2024-11-18 16:53:13,167] A new study created in memory with name: no-name-0c07d24e-5cce-4a3c-ac7b-c894c0e0f980
[I 2024-11-18 16:53:13,173] Trial 0 finished with value: 0.8462290625317939 and parameters: {'feature_0': 6.6544593171473645, 'feature_1': 9.62913071360715, 'feature_2': 69.30841175580225, 'feature_3': 32.21173954593593, 'feature_4': 86.96935193371581, 'feature_5': 370.5827464856758}. Best is trial 0 with value: 0.8462290625317939.
[I 2024-11-18 16:53:13,177] Trial 1 finished with value: 4.435345580590749 and parameters: {'feature_0': 5.960300355424834, 'feature_1': 9.82590157426688, 'feature_2': 29.28447756140389, 'feature_3': 16.426023382591687, 'feature_4': 31.44456433870318, 'feature_5': 305.239251233495}. Best is trial 0 with value: 0.8462290625317939.
[I 2024-11-18 16:53:13,182] Trial 2 finished with value: 0.7130960227557828 and parameters: {'feature_0': 14.923789459460789, 'feature_1': 7.515094059580155, 'feature_2': 85.6871174825057, 'feature_3': 37.16976934776303,

Optimal Features: dict_values([14.264914156452802, 5.727508783132752, 58.258287978545845, 29.221397025179773, 94.0965661033762, 366.65731070206266])
Minimum Combined Error: 0.09308502823669262


(dict_values([14.264914156452802, 5.727508783132752, 58.258287978545845, 29.221397025179773, 94.0965661033762, 366.65731070206266]),
 0.09308502823669262)